# INDIAI - ARTIFICIAL INTELLIGENCE PROJECT


### Group : `Manav Thontia`     |   `Delfina Ferreri`    |  `Mathis Veschambre`  |   `Shiva TARDIVEL` 

# INTRODUCTION 

- *Context of the problem*
- *We will be wotking with five sources of data in csv format*

# DATA PREPROCESSING
Setting the environment

In [88]:
import numpy as np
import os
from numpy.random import default_rng
import matplotlib.pyplot as plt
import pandas as pd

## Loading the DataSets

In [89]:
# os.chdir("Change this to the relative/absolute path of the Datasets folder")
os.chdir("./Datasets")

general_data = pd.read_csv('general_data.csv').copy()
employee_survey_data = pd.read_csv('employee_survey_data.csv').copy()
manager_survey_data = pd.read_csv('manager_survey_data.csv').copy()
in_time = pd.read_csv('in_time.csv').copy()
out_time = pd.read_csv('out_time.csv').copy()

os.chdir("..")

The datasets will be processed separately, then once clean thet'll be merged into one.

## Before merging all in one

### General Data

In [90]:
general_data.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Columns: 24 entries, Age to YearsWithCurrManager
dtypes: float64(2), int64(14), object(8)
memory usage: 827.0+ KB


What can be observed from general data so far?
- Data from 4410 employees was gathered.
- Most of the attributes -exactly 14- are numerical, accepting either integers or floats as values.
- The other 5 attributes seem to be categorical.

Let's take a look at what the general data looks like:

In [91]:
general_data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,...,4.0,Y,12,8,2,9.0,2,6,0,4


#### Dropping unnecesary attributes

Just by looking, we notice that:
- The attribute **Over18** is redundant: the more precise attribute **Age** is also present.
- The attribute **EmployeeCount** makes no sense, as each entry represents only one employee.
- We can divise two attributes that represent sensitive personal information, like **Gender** and **MaritalStatus**. We don't consider these relevant for the current analysis.

Let's now look at the metrics for each numerical attribute.

In [92]:
general_data.describe()

,Age,DistanceFromHome,Education,EmployeeCount,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
count,4410.000000,4410.000000,4410.000000,4410.0,4410.000000,4410.000000,4410.000000,4391.000000,4410.000000,4410.0,4410.000000,4401.000000,4410.000000,4410.000000,4410.000000,4410.000000
mean,36.923810,9.192517,2.912925,1.0,2205.500000,2.063946,65029.312925,2.694830,15.209524,8.0,0.793878,11.279936,2.799320,7.008163,2.187755,4.123129
std,9.133301,8.105026,1.023933,0.0,1273.201673,1.106689,47068.888559,2.498887,3.659108,0.0,0.851883,7.782222,1.288978,6.125135,3.221699,3.567327
min,18.000000,1.000000,1.000000,1.0,1.000000,1.000000,10090.000000,0.000000,11.000000,8.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,2.000000,2.000000,1.0,1103.250000,1.000000,29110.000000,1.000000,12.000000,8.0,0.000000,6.000000,2.000000,3.000000,0.000000,2.000000
50%,36.000000,7.000000,3.000000,1.0,2205.500000,2.000000,49190.000000,2.000000,14.000000,8.0,1.000000,10.000000,3.000000,5.000000,1.000000,3.000000
75%,43.000000,14.000000,4.000000,1.0,3307.750000,3.000000,83800.000000,4.000000,18.000000,8.0,1.000000,15.000000,3.000000,9.000000,3.000000,7.000000
max,60.000000,29.000000,5.000000,1.0,4410.000000,5.000000,199990.000000,9.000000,25.000000,8.0,3.000000,40.000000,6.000000,40.000000,15.000000,17.000000


- The attributes **EmployeeCount** and **StandardHours** have a standard deviation of 0.0, meaning that, for them, all entries have the same value -which are 1 and 8.0 respectively-. They are not useful to tell employees apart in any way.

We proceed by dropping the mentioned attributes, reducing the number of columns from 24 to 19.

In [93]:
general_data.drop('Gender', axis=1,inplace=True)
general_data.drop('MaritalStatus', axis=1,inplace=True)
general_data.drop('Over18', axis=1,inplace=True)
general_data.drop('EmployeeCount', axis=1,inplace=True)
general_data.drop('StandardHours', axis=1,inplace=True)
general_data

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,JobLevel,JobRole,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Healthcare Representative,131160,1.0,11,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,2,1,Research Scientist,41890,0.0,23,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,3,4,Sales Executive,193280,1.0,15,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,4,3,Human Resources,83210,3.0,11,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,5,1,Sales Executive,23420,4.0,12,2,9.0,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42,No,Travel_Rarely,Research & Development,5,4,Medical,4406,1,Research Scientist,60290,3.0,17,1,10.0,5,3,0,2
4406,29,No,Travel_Rarely,Research & Development,2,4,Medical,4407,1,Laboratory Technician,26790,2.0,15,0,10.0,2,3,0,2
4407,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,4408,2,Sales Executive,37020,0.0,20,0,5.0,4,4,1,2
4408,42,No,Travel_Rarely,Sales,18,2,Medical,4409,1,Laboratory Technician,23980,0.0,14,1,10.0,2,9,7,8


### Survey Data: Employee and Manager

These datasets contain information about the employee's feelings and performance at work, both from it's point of view and from the point of view of its manager. At first sight, all attributes seem like they could be relevant for the current analysis.

In [94]:
employee_survey_data.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


In [95]:
manager_survey_data.head()

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


Given their similarity, it seems natural to merge these two into one dataset, using **EmployeeID** as the key attribute.

In [96]:
survey_data = pd.merge(employee_survey_data,manager_survey_data,on="EmployeeID",how="outer")
survey_data.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,3.0,4.0,2.0,3,3
1,2,3.0,2.0,4.0,2,4
2,3,2.0,2.0,1.0,3,3
3,4,4.0,4.0,3.0,2,3
4,5,4.0,1.0,3.0,3,3


In [97]:
survey_data.describe()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
count,4410.000000,4385.000000,4390.000000,4372.000000,4410.000000,4410.000000
mean,2205.500000,2.723603,2.728246,2.761436,2.729932,3.153741
std,1273.201673,1.092756,1.101253,0.706245,0.711400,0.360742
min,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000
25%,1103.250000,2.000000,2.000000,2.000000,2.000000,3.000000
50%,2205.500000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3307.750000,4.000000,4.000000,3.000000,3.000000,3.000000
max,4410.000000,4.000000,4.000000,4.000000,4.000000,4.000000


### Schedule Data: In and Out Times

These datasets contain extremely detailed information on the times of entry and exit of each employee during the last 250 week-days.

Extra processing will be needed for these two sets of data, as this information on its own is not useful yet.

What new attributes for each employee could we extract from here?
- **Average of hours worked per day** (without taking into consideration those days when the employee was absent)
- **Number of absences** (days where there is no recorded time for entry and exit of the employee)
- **Schedule inconsistency** (how much the amount of hours worked each day varies, standard deviation for working hours)
- **Punctuality at entry** (standard deviation for entry time) - not anymore
- **Punctuality at exit** (standard deviation for exit time) - not anymore

But first, small peek at what the data looks like:

In [98]:
in_time.head()

,Unnamed: 0,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,NaN,2015-01-02 09:43:45,2015-01-05 10:08:48,2015-01-06 09:54:26,2015-01-07 09:34:31,2015-01-08 09:51:09,2015-01-09 10:09:25,2015-01-12 09:42:53,2015-01-13 10:13:06,...,NaN,2015-12-21 09:55:29,2015-12-22 10:04:06,2015-12-23 10:14:27,2015-12-24 10:11:35,NaN,2015-12-28 10:13:41,2015-12-29 10:03:36,2015-12-30 09:54:12,2015-12-31 10:12:44
1,2,NaN,2015-01-02 10:15:44,2015-01-05 10:21:05,NaN,2015-01-07 09:45:17,2015-01-08 10:09:04,2015-01-09 09:43:26,2015-01-12 10:00:07,2015-01-13 10:43:29,...,2015-12-18 10:37:17,2015-12-21 09:49:02,2015-12-22 10:33:51,2015-12-23 10:12:10,NaN,NaN,2015-12-28 09:31:45,2015-12-29 09:55:49,2015-12-30 10:32:25,2015-12-31 09:27:20
2,3,NaN,2015-01-02 10:17:41,2015-01-05 09:50:50,2015-01-06 10:14:13,2015-01-07 09:47:27,2015-01-08 10:03:40,2015-01-09 10:05:49,2015-01-12 10:03:47,2015-01-13 10:21:26,...,2015-12-18 10:15:14,2015-12-21 10:10:28,2015-12-22 09:44:44,2015-12-23 10:15:54,2015-12-24 10:07:26,NaN,2015-12-28 09:42:05,2015-12-29 09:43:36,2015-12-30 09:34:05,2015-12-31 10:28:39
3,4,NaN,2015-01-02 10:05:06,2015-01-05 09:56:32,2015-01-06 10:11:07,2015-01-07 09:37:30,2015-01-08 10:02:08,2015-01-09 10:08:12,2015-01-12 10:13:42,2015-01-13 09:53:22,...,2015-12-18 10:17:38,2015-12-21 09:58:21,2015-12-22 10:04:25,2015-12-23 10:11:46,2015-12-24 09:43:15,NaN,2015-12-28 09:52:44,2015-12-29 09:33:16,2015-12-30 10:18:12,2015-12-31 10:01:15
4,5,NaN,2015-01-02 10:28:17,2015-01-05 09:49:58,2015-01-06 09:45:28,2015-01-07 09:49:37,2015-01-08 10:19:44,2015-01-09 10:00:50,2015-01-12 10:29:27,2015-01-13 09:59:32,...,2015-12-18 09:58:35,2015-12-21 10:03:41,2015-12-22 10:10:30,2015-12-23 10:13:36,2015-12-24 09:44:24,NaN,2015-12-28 10:05:15,2015-12-29 10:30:53,2015-12-30 09:18:21,2015-12-31 09:41:09


In [99]:
out_time.head()

,Unnamed: 0,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,NaN,2015-01-02 16:56:15,2015-01-05 17:20:11,2015-01-06 17:19:05,2015-01-07 16:34:55,2015-01-08 17:08:32,2015-01-09 17:38:29,2015-01-12 16:58:39,2015-01-13 18:02:58,...,NaN,2015-12-21 17:15:50,2015-12-22 17:27:51,2015-12-23 16:44:44,2015-12-24 17:47:22,NaN,2015-12-28 18:00:07,2015-12-29 17:22:30,2015-12-30 17:40:56,2015-12-31 17:17:33
1,2,NaN,2015-01-02 18:22:17,2015-01-05 17:48:22,NaN,2015-01-07 17:09:06,2015-01-08 17:34:04,2015-01-09 16:52:29,2015-01-12 17:36:48,2015-01-13 18:00:13,...,2015-12-18 18:31:28,2015-12-21 17:34:16,2015-12-22 18:16:35,2015-12-23 17:38:18,NaN,NaN,2015-12-28 17:08:38,2015-12-29 17:54:46,2015-12-30 18:31:35,2015-12-31 17:40:58
2,3,NaN,2015-01-02 16:59:14,2015-01-05 17:06:46,2015-01-06 16:38:32,2015-01-07 16:33:21,2015-01-08 17:24:22,2015-01-09 16:57:30,2015-01-12 17:28:54,2015-01-13 17:21:25,...,2015-12-18 17:02:23,2015-12-21 17:20:17,2015-12-22 16:32:50,2015-12-23 16:59:43,2015-12-24 16:58:25,NaN,2015-12-28 16:43:31,2015-12-29 17:09:56,2015-12-30 17:06:25,2015-12-31 17:15:50
3,4,NaN,2015-01-02 17:25:24,2015-01-05 17:14:03,2015-01-06 17:07:42,2015-01-07 16:32:40,2015-01-08 16:53:11,2015-01-09 17:19:47,2015-01-12 17:13:37,2015-01-13 17:11:45,...,2015-12-18 17:55:23,2015-12-21 16:49:09,2015-12-22 17:24:00,2015-12-23 17:36:35,2015-12-24 16:48:21,NaN,2015-12-28 17:19:34,2015-12-29 16:58:16,2015-12-30 17:40:11,2015-12-31 17:09:14
4,5,NaN,2015-01-02 18:31:37,2015-01-05 17:49:15,2015-01-06 17:26:25,2015-01-07 17:37:59,2015-01-08 17:59:28,2015-01-09 17:44:08,2015-01-12 18:51:21,2015-01-13 18:14:58,...,2015-12-18 17:52:48,2015-12-21 17:43:35,2015-12-22 18:07:57,2015-12-23 18:00:49,2015-12-24 17:59:22,NaN,2015-12-28 17:44:59,2015-12-29 18:47:00,2015-12-30 17:15:33,2015-12-31 17:42:14


#### Dropping unnecesary attributes

- Some days seem to have N/A values for all employees! Among these, the first day of the year and Christmas.

- These non-workable days are not relevant for our analysis, as they don't provide information to tell employees apart.

- We proceed by identifying them and dropping the columns that match our description.

In [100]:
nonsence_entries_in = pd.DataFrame(in_time.loc[:, in_time.isnull().sum() == len(in_time) ])
print(nonsence_entries_in.shape[1]," days with all null values for time of entrance.")
nonsence_entries_in

12  days with all null values for time of entrance.


,2015-01-01,2015-01-14,2015-01-26,2015-03-05,2015-05-01,2015-07-17,2015-09-17,2015-10-02,2015-11-09,2015-11-10,2015-11-11,2015-12-25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
nonsence_entries_out = pd.DataFrame(out_time.loc[:, in_time.isnull().sum() == len(in_time)])
print(nonsence_entries_out.shape[1]," days with all null values for time of exit.")
nonsence_entries_out

12  days with all null values for time of exit.


,2015-01-01,2015-01-14,2015-01-26,2015-03-05,2015-05-01,2015-07-17,2015-09-17,2015-10-02,2015-11-09,2015-11-10,2015-11-11,2015-12-25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


It is interesting to verify if the dates of non-workable days are consistent, meaning in time and out time data are well synchronized.

In [102]:
print("The non-workable days in in_data are the same as in out_data?: ",nonsence_entries_in.equals(nonsence_entries_out))

The non-workable days in in_data are the same as in out_data?:  True


In [103]:
in_time = in_time.drop(columns=nonsence_entries_in.columns)
out_time = out_time.drop(columns=nonsence_entries_out.columns)

By retiring the non-workable days, we reduced the columns with missing values by 12. Now let's look at other cases of this phenomena.

#### Treatment of missing values

Before acting on missing values, a function to quickly detect them:

In [104]:
def DisplayMissingValues(data, data_name):
    
    df = pd.DataFrame(data)

    missing_counts = df.isnull().sum()
    missing_columns = missing_counts[missing_counts > 0]
    
    if missing_columns.empty:
        print("[0] No missing values in:",data_name,"\n")
        return
    print(len(missing_columns)," attributes with missing values in: ",data_name)
    print(missing_columns.to_string(),"\n")

In [105]:
DisplayMissingValues(in_time, "in time")

249  attributes with missing values in:  in time
2015-01-02    209
2015-01-05    206
2015-01-06    228
2015-01-07    209
2015-01-08    243
2015-01-09    215
2015-01-12    239
2015-01-13    265
2015-01-15    236
2015-01-16    264
2015-01-19    228
2015-01-20    241
2015-01-21    240
2015-01-22    240
2015-01-23    249
2015-01-27    217
2015-01-28    212
2015-01-29    276
2015-01-30    208
2015-02-02    208
2015-02-03    206
2015-02-04    212
2015-02-05    211
2015-02-06    211
2015-02-09    224
2015-02-10    219
2015-02-11    226
2015-02-12    236
2015-02-13    212
2015-02-16    222
2015-02-17    244
2015-02-18    229
2015-02-19    217
2015-02-20    212
2015-02-23    241
2015-02-24    226
2015-02-25    234
2015-02-26    238
2015-02-27    236
2015-03-02    254
2015-03-03    228
2015-03-04    231
2015-03-06    242
2015-03-09    232
2015-03-10    243
2015-03-11    232
2015-03-12    219
2015-03-13    191
2015-03-16    219
2015-03-17    231
2015-03-18    237
2015-03-19    239
2015-03-20    2

In [106]:

DisplayMissingValues(out_time, "out time")

249  attributes with missing values in:  out time
2015-01-02    209
2015-01-05    206
2015-01-06    228
2015-01-07    209
2015-01-08    243
2015-01-09    215
2015-01-12    239
2015-01-13    265
2015-01-15    236
2015-01-16    264
2015-01-19    228
2015-01-20    241
2015-01-21    240
2015-01-22    240
2015-01-23    249
2015-01-27    217
2015-01-28    212
2015-01-29    276
2015-01-30    208
2015-02-02    208
2015-02-03    206
2015-02-04    212
2015-02-05    211
2015-02-06    211
2015-02-09    224
2015-02-10    219
2015-02-11    226
2015-02-12    236
2015-02-13    212
2015-02-16    222
2015-02-17    244
2015-02-18    229
2015-02-19    217
2015-02-20    212
2015-02-23    241
2015-02-24    226
2015-02-25    234
2015-02-26    238
2015-02-27    236
2015-03-02    254
2015-03-03    228
2015-03-04    231
2015-03-06    242
2015-03-09    232
2015-03-10    243
2015-03-11    232
2015-03-12    219
2015-03-13    191
2015-03-16    219
2015-03-17    231
2015-03-18    237
2015-03-19    239
2015-03-20    

Too many missing values!
A treatment protocol must be chosen.

First, let's verify there are no _inconsistencies_ in the missing values. 

What do we mean by inconsistencies? Days where, for certain employee, either entry time or exit time is missing, but not both at the same time.
An occurrence of this kind would be a problem, as it would clearly not make sense for an employee to get in and then never get out, or viceversa.

In [107]:
null_inconsistency_df = in_time.isnull() ^ out_time.isnull()    # XOR operation for each cell
print("The number of null inconsistencies is: ",(null_inconsistency_df == True).sum().sum())    # Counting inconsistencies

The number of null inconsistencies is:  0


No inconsistencies. This means in time and out time data are well synchronized.

Then for the days both entry and exit time are missing -which, as we have just observed, happens at least once for each recorded day- the decision to take it as an absent day, with zero hours of work, was taken. 

It's unknown to us the reason why the employee didn't attend work that day, so a neutral enough name was picked for this new attribute: **TotalAbsences**.

In [108]:
def minutes_since_zero(input_time):
    return input_time.hour * 60 + input_time.minute

In [109]:
# change values into minutes from earliest time
# Convert to datetime format (if it's not already)
in_time = in_time.apply(pd.to_datetime)
in_time = in_time.applymap(minutes_since_zero)
in_time

# Extract only the time component (HH:MM:SS)
#in_time['time_only'] = in_time['datetime_column'].dt.time

/var/folders/vw/cl6_2pns2m5f8sp50m3z244m0000gn/T/ipykernel_57720/1286777584.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  in_time = in_time.applymap(minutes_since_zero)


,Unnamed: 0,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-15,...,2015-12-17,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,0,583.0,608.0,594.0,574.0,591.0,609.0,582.0,613.0,601.0,...,NaN,NaN,595.0,604.0,614.0,611.0,613.0,603.0,594.0,612.0
1,0,615.0,621.0,NaN,585.0,609.0,583.0,600.0,643.0,577.0,...,555.0,637.0,589.0,633.0,612.0,NaN,571.0,595.0,632.0,567.0
2,0,617.0,590.0,614.0,587.0,603.0,605.0,603.0,621.0,595.0,...,593.0,615.0,610.0,584.0,615.0,607.0,582.0,583.0,574.0,628.0
3,0,605.0,596.0,611.0,577.0,602.0,608.0,613.0,593.0,600.0,...,594.0,617.0,598.0,604.0,611.0,583.0,592.0,573.0,618.0,601.0
4,0,628.0,589.0,585.0,589.0,619.0,600.0,629.0,599.0,606.0,...,586.0,598.0,603.0,610.0,613.0,584.0,605.0,630.0,558.0,581.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,0,560.0,617.0,626.0,606.0,585.0,589.0,577.0,565.0,569.0,...,605.0,601.0,625.0,616.0,604.0,585.0,615.0,610.0,568.0,600.0
4406,0,603.0,NaN,584.0,582.0,600.0,584.0,607.0,605.0,618.0,...,585.0,567.0,581.0,590.0,632.0,587.0,594.0,613.0,621.0,609.0
4407,0,601.0,573.0,589.0,628.0,587.0,601.0,589.0,587.0,608.0,...,601.0,600.0,591.0,602.0,598.0,596.0,599.0,NaN,602.0,603.0
4408,0,617.0,602.0,612.0,612.0,582.0,NaN,600.0,588.0,544.0,...,591.0,594.0,601.0,610.0,582.0,596.0,595.0,594.0,615.0,596.0


In [110]:
out_time = out_time.apply(pd.to_datetime)
out_time.applymap(minutes_since_zero) 

/var/folders/vw/cl6_2pns2m5f8sp50m3z244m0000gn/T/ipykernel_57720/744982645.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  out_time.applymap(minutes_since_zero)


,Unnamed: 0,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-15,...,2015-12-17,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,0,1016.0,1040.0,1039.0,994.0,1028.0,1058.0,1018.0,1082.0,1042.0,...,NaN,NaN,1035.0,1047.0,1004.0,1067.0,1080.0,1042.0,1060.0,1037.0
1,0,1102.0,1068.0,NaN,1029.0,1054.0,1012.0,1056.0,1080.0,1034.0,...,1030.0,1111.0,1054.0,1096.0,1058.0,NaN,1028.0,1074.0,1111.0,1060.0
2,0,1019.0,1026.0,998.0,993.0,1044.0,1017.0,1048.0,1041.0,1041.0,...,1026.0,1022.0,1040.0,992.0,1019.0,1018.0,1003.0,1029.0,1026.0,1035.0
3,0,1045.0,1034.0,1027.0,992.0,1013.0,1039.0,1033.0,1031.0,1013.0,...,1041.0,1075.0,1009.0,1044.0,1056.0,1008.0,1039.0,1018.0,1060.0,1029.0
4,0,1111.0,1069.0,1046.0,1057.0,1079.0,1064.0,1131.0,1094.0,1101.0,...,1085.0,1072.0,1063.0,1087.0,1080.0,1079.0,1064.0,1127.0,1035.0,1062.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,0,1047.0,1148.0,1130.0,1137.0,1078.0,1086.0,1078.0,1090.0,1070.0,...,1115.0,1086.0,1115.0,1113.0,1120.0,1101.0,1124.0,1154.0,1104.0,1110.0
4406,0,979.0,NaN,907.0,925.0,972.0,926.0,970.0,982.0,979.0,...,944.0,923.0,931.0,945.0,998.0,947.0,934.0,1007.0,963.0,978.0
4407,0,1037.0,1028.0,1047.0,1107.0,1025.0,1022.0,1055.0,1035.0,1095.0,...,1062.0,1068.0,1063.0,1067.0,1063.0,1040.0,1063.0,NaN,1068.0,1088.0
4408,0,1188.0,1177.0,1200.0,1175.0,1135.0,NaN,1158.0,1164.0,1113.0,...,1146.0,1192.0,1161.0,1172.0,1137.0,1177.0,1198.0,1135.0,1177.0,1173.0


In [111]:
def nan_to_zero(value):
    if pd.isna(value):
        value = 0
    return value

In [112]:
in_time = in_time.applymap(nan_to_zero)
in_time

/var/folders/vw/cl6_2pns2m5f8sp50m3z244m0000gn/T/ipykernel_57720/1538203280.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  in_time = in_time.applymap(nan_to_zero)


,Unnamed: 0,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-15,...,2015-12-17,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,0,583.0,608.0,594.0,574.0,591.0,609.0,582.0,613.0,601.0,...,0.0,0.0,595.0,604.0,614.0,611.0,613.0,603.0,594.0,612.0
1,0,615.0,621.0,0.0,585.0,609.0,583.0,600.0,643.0,577.0,...,555.0,637.0,589.0,633.0,612.0,0.0,571.0,595.0,632.0,567.0
2,0,617.0,590.0,614.0,587.0,603.0,605.0,603.0,621.0,595.0,...,593.0,615.0,610.0,584.0,615.0,607.0,582.0,583.0,574.0,628.0
3,0,605.0,596.0,611.0,577.0,602.0,608.0,613.0,593.0,600.0,...,594.0,617.0,598.0,604.0,611.0,583.0,592.0,573.0,618.0,601.0
4,0,628.0,589.0,585.0,589.0,619.0,600.0,629.0,599.0,606.0,...,586.0,598.0,603.0,610.0,613.0,584.0,605.0,630.0,558.0,581.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,0,560.0,617.0,626.0,606.0,585.0,589.0,577.0,565.0,569.0,...,605.0,601.0,625.0,616.0,604.0,585.0,615.0,610.0,568.0,600.0
4406,0,603.0,0.0,584.0,582.0,600.0,584.0,607.0,605.0,618.0,...,585.0,567.0,581.0,590.0,632.0,587.0,594.0,613.0,621.0,609.0
4407,0,601.0,573.0,589.0,628.0,587.0,601.0,589.0,587.0,608.0,...,601.0,600.0,591.0,602.0,598.0,596.0,599.0,0.0,602.0,603.0
4408,0,617.0,602.0,612.0,612.0,582.0,0.0,600.0,588.0,544.0,...,591.0,594.0,601.0,610.0,582.0,596.0,595.0,594.0,615.0,596.0


In [131]:

##

time_data = pd.DataFrame(index=in_time.index, columns=['total_time_worked', 'mean_time_worked', 'total_working_days'])
# Function to calculate total hours worked for each day (in hours) and mean time
def calculate_working_hours(row):
    total_hours = 0
    working_hours_for_each_employee = []

    for date in in_time.columns[1:]:    # Skipping EmployeeID column
        check_in = row[date + '_x']     # Intime
        check_out = row[date + '_y']    # Outtime
        hours_worked = (check_out - check_in)
        total_hours += hours_worked
        working_hours_for_each_employee.append(hours_worked)

    mean_time = np.mean(working_hours_for_each_employee) if working_hours_for_each_employee else 0
    return total_hours, mean_time

# Ensure there are rows in time_data before applying calculations
if not time_data.empty:
    # Add total time worked and mean time worked columns
    time_data.iloc[1:][['total_time_worked', 'mean_time_worked']] = time_data.iloc[1:].apply(
        lambda row: pd.Series(calculate_working_hours(row)), axis=1
    )
    # Calculate total working days for each employee
    time_data['total_working_days'] = time_data.apply(
        lambda row: sum(pd.notnull(row[date + '_x']) and pd.notnull(row[date + '_y']) for date in in_time.columns[1:]), axis=1
    )

    # Convert to integers
    time_data['total_time_worked'] = time_data['total_time_worked'].astype(int)
    time_data['mean_time_worked'] = time_data['mean_time_worked'].astype(int)

    # Show result
    print(time_data)
else:
    print("The time_data DataFrame is empty.")

time_data
#final_data = pd.merge(full_data, merged_data, on="EmployeeID", how="left")

#columns_to_drop = [col for col in final_data.columns if '_x' in col or '_y' in col]
#final_data = final_data.drop(columns=columns_to_drop)

# Save the cleaned dataset
#final_data.to_csv('data_final.csv', index=False)

#copy3 = final_data.copy()
#copy3.head()


KeyError: '2015-01-02_x'

Now the two will be combined into one dataset containing the previously mentionned attributes, and using **EmployeeID** as the key.

In [113]:
# SHIVA'S CODE

# incorporate new metric: standard deviation

time_data = pd.DataFrame(columns=["EmployeeID"])

## Merging all into one dataset

It is time to combine all the data into one unique dataset, which we will call **merged_data**.

After that, we will assemble the full pipeline to properly finish the preprocessing of the data from this point on.

In [114]:
merge_key = "EmployeeID"
merged_data = pd.merge(general_data, survey_data, on=merge_key, how="outer")
merged_data = pd.merge(merged_data, time_data, on=merge_key, how="outer")       # time data is missing

# we rearrange the order of the columns for an easier read
columns_order = ["EmployeeID"] + ["Attrition"] + [col for col in merged_data.columns if (col != "EmployeeID" and col != "Attrition")] 
merged_df = merged_data[columns_order]
merged_df.to_csv("merged_data.csv", index=True)
merged_df.head(7)


,EmployeeID,Attrition,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,JobLevel,JobRole,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,No,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Healthcare Representative,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,2,Yes,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Research Scientist,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,3,No,32,Travel_Frequently,Research & Development,17,4,Other,4,Sales Executive,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,4,No,38,Non-Travel,Research & Development,2,5,Life Sciences,3,Human Resources,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,5,No,32,Travel_Rarely,Research & Development,10,1,Medical,1,Sales Executive,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3
5,6,No,46,Travel_Rarely,Research & Development,8,3,Life Sciences,4,Research Director,...,28.0,5,7,7,7,3.0,2.0,2.0,3,3
6,7,Yes,28,Travel_Rarely,Research & Development,11,2,Medical,2,Sales Executive,...,5.0,2,0,0,0,1.0,3.0,1.0,3,4


## Assembling the full pipeline

In [115]:
merged_data.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Columns: 24 entries, Age to PerformanceRating
dtypes: float64(5), int64(14), object(5)
memory usage: 827.0+ KB


We should prepare the pipeline for:
- Treating missing values,
- Treating categorical values (appropriate encoding),
- Treating outliers,
- Standarizing if necessary.

### Treatment of missing values

In [116]:

DisplayMissingValues(merged_df,"merged data")

5  attributes with missing values in:  merged data
NumCompaniesWorked         19
TotalWorkingYears           9
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38 



In [117]:
# SHIVA'S CODE
# MANAV'S CODE

### Treating categorical values

In [118]:
# SHIVA'S CODE (ENCODING)

2 attributes with missing values

In [119]:
def GetTime(datetime):
    if pd.notnull(datetime):
        return datetime[11:16]
    return None

def TimeToMinutes(time):
    if pd.notnull(time):
        return int(time[0:2])*60 + int(time[4:6])
    return 0

In [120]:
working_hours = pd.DataFrame(columns=["EmployeeID","AvgWorkingHours","RecentInasistances"])
for id in range (1,total_employees):
    # para calcular el promedio: procuro que todo este ordenado por id del mas bajo al mas alto
    # voy contando las inasistencias
    in_time[1:][id].sum()
    out_time[1:][id]
    avg_working_hours = round(round((TimeToMinutes(out_t)-TimeToMinutes(in_t))/60,2).sum()/emp["WORKED_HOURS"].ne(0).sum(),2)
    total_inasistances = emp["WORKED_HOURS"].eq(0).sum()
    working_hours.loc[len(working_hours)] = (id,avg_working_hours,total_inasistances)

NameError: name 'total_employees' is not defined

In [ ]:
# don't run this again. It took me ONE HOUR.
def EmployeesWorkingHoursInfo_DF(in_time,out_time,ID):
    
    wh_info = pd.DataFrame(columns=['DATE','IN_TIME','OUT_TIME','WORKED_HOURS'])

    employee_in_time_data = in_time[in_time['Unnamed: 0']==ID]      # all entry times for given employee
    employee_out_time_data = out_time[out_time['Unnamed: 0']==ID]   # all exit times for given employee

    for i in range (1,len(employee_in_time_data.columns)):  # 250 iterations: all days employee worked
        in_t    = GetTime(employee_in_time_data[employee_in_time_data.columns[i]][ID-1])
        out_t   = GetTime(employee_out_time_data[employee_out_time_data.columns[i]][ID-1])
        wh_info.loc[len(wh_info)] = (employee_in_time_data.columns[i],in_t,out_t,round((TimeToMinutes(out_t)-TimeToMinutes(in_t))/60,2))

    return wh_info

info_emp_3 = EmployeesWorkingHoursInfo_DF(in_time,out_time,4400)
print("TOTAL WORKABLE DAYS:               ",max(info_emp_3.count()))
print("AVERAGE WORKING HOURS PER WORK DAY:",round(info_emp_3["WORKED_HOURS"].sum()/info_emp_3["WORKED_HOURS"].ne(0).sum(),2))
print("AMOUNT OF RECENT INASSISTANCES:    ",info_emp_3["WORKED_HOURS"].eq(0).sum())

# posibles nuevos atributos: avg worked hours, missed days, standard deviation as in punctuality at entry or outing?
working_hours = pd.DataFrame(columns=["EmployeeID","AvgWorkingHours","RecentInasistances"])
for id in range (1,total_employees):
    emp = EmployeesWorkingHoursInfo_DF(in_time,out_time,id)
    avg_working_hours = round(emp["WORKED_HOURS"].sum()/emp["WORKED_HOURS"].ne(0).sum(),2)
    total_inasistances = emp["WORKED_HOURS"].eq(0).sum()
    working_hours.loc[len(working_hours)] = (id,avg_working_hours,total_inasistances)


TOTAL WORKABLE DAYS:                249
AVERAGE WORKING HOURS PER WORK DAY: 6.36
AMOUNT OF RECENT INASSISTANCES:     22


In [ ]:

# Placeholder for your datasets (replace with your actual loading code if different)
in_time = pd.read_csv('in_time.csv'); out_time = pd.read_csv('out_time.csv')
# For this example, I assume in_time and out_time are already loaded DataFrames

# Step 1: Function to parse time strings into minutes
def parse_time(s):
    if pd.isna(s) or s == '' or not isinstance(s, str):
        return np.nan
    try:
        h, m = s.split(':')
        return int(h) * 60 + int(m)
    except:
        return np.nan

# Step 2: Identify day columns (exclude the EmployeeID column)
day_columns = in_time.columns[1:]  # Assuming first column is 'Unnamed: 0' or similar

# Step 3: Convert all time entries to minutes
in_time_minutes = in_time[day_columns].applymap(parse_time)
out_time_minutes = out_time[day_columns].applymap(parse_time)

# Step 4: Calculate worked hours per day (in hours)
worked_hours = (out_time_minutes - in_time_minutes) / 60

# Step 5: Compute total worked hours per employee
total_worked_hours = worked_hours.sum(axis=1, skipna=True)

# Step 6: Count days where the employee actually worked (hours > 0 and not NaN)
worked_days = ((worked_hours > 0) & worked_hours.notna()).sum(axis=1)

# Step 7: Calculate average working hours per worked day
# If no days worked, set to 0 to avoid division by zero
avg_working_hours = np.where(worked_days > 0, total_worked_hours / worked_days, 0)

# Step 8: Calculate inassistances (days with no work or missing data)
inassistances = (worked_hours == 0) | worked_hours.isna()
total_inassistances = inassistances.sum(axis=1)

# Step 9: Create the result DataFrame
working_hours = pd.DataFrame({
    "EmployeeID": in_time['Unnamed: 0'],
    "AvgWorkingHours": np.round(avg_working_hours, 2),
    "RecentInasistances": total_inassistances
})

# Step 10: Display general statistics
total_workable_days = len(day_columns)  # Total days in the dataset
print("TOTAL WORKABLE DAYS:               ", total_workable_days)
print("AVERAGE WORKING HOURS PER WORK DAY:", round(working_hours['AvgWorkingHours'].mean(), 2))
print("AMOUNT OF RECENT INASSISTANCES:    ", working_hours['RecentInasistances'].sum())

# Step 11: Display results for all employees
print("\nWorking hours for all employees:")
print(working_hours.head(6))  # Show first 6 employees to match your output

No missing values

In [ ]:
emp = EmployeesWorkingHoursInfo_DF(in_time,out_time,4410)
avg_working_hours = round(emp["WORKED_HOURS"].sum()/emp["WORKED_HOURS"].ne(0).sum(),2)
total_inasistances = emp["WORKED_HOURS"].eq(0).sum()
working_hours.loc[len(working_hours)] = (4410,avg_working_hours,total_inasistances)

In [ ]:
copy_working_hours = working_hours.copy()
copy_working_hours["EmployeeID"] = copy_working_hours["EmployeeID"].astype(int)
copy_working_hours["RecentInasistances"] = copy_working_hours["RecentInasistances"].astype(int)
copy_working_hours

,EmployeeID,AvgWorkingHours,RecentInasistances
0,1,7.41,17
1,2,7.75,13
2,3,6.99,7
3,4,7.22,14
4,5,8.04,4
...,...,...,...
4405,4406,8.50,6
4406,4407,6.11,8
4407,4408,7.72,18
4411,4409,9.46,8


Merge all datasets into one